# 🔍 Diff Analysis - Demo

Este notebook demonstra como usar o **Copilot-IA** para analisar diferenças de código (diffs).

## Funcionalidades

- 📝 Análise de diffs/patches
- 🔄 Comparação entre versões de código
- 🐛 Detecção de bugs introduzidos
- 🔒 Identificação de problemas de segurança
- 💡 Sugestões de melhoria para as mudanças


## 1. Setup Inicial


In [1]:
# Configuração do ambiente
import sys
sys.path.insert(0, '..')

from src.utils import setup_logging
setup_logging(level="INFO")

print("✅ Setup concluído!")


✅ Setup concluído!


In [2]:
# Importar os módulos necessários
from src.copilots import CodeReviewerCopilot
from src.tools.git_tools import GitTool

# Inicializar o Code Reviewer
reviewer = CodeReviewerCopilot()

# Inicializar ferramenta Git (opcional)
git_tool = GitTool()

print(f"🔍 Code Reviewer inicializado!")
print(f"   Modelo: {reviewer.model_config.name}")
print(f"   Provider: {reviewer.model_config.provider}")


[12/11/25 16:00:49] INFO     2025-12-11 16:00:49,655 - src.agents.base - INFO - Copiloto 'Code Reviewer'           
                             inicializado com openai/gpt-4o

                    WARNING  2025-12-11 16:00:49,812 - src.tools.git_tools - WARNING - Não foi possível inicializar
                             repositório Git: /home/zero/Workspace/Copilot-ia/notebooks

🔍 Code Reviewer inicializado!
   Modelo: gpt-4o
   Provider: openai


## 2. Analisar um Diff Simples

Vamos analisar uma mudança que introduz um bug.


In [3]:
# Diff de exemplo - introduz bug de divisão por zero
diff_bugado = '''
--- a/calculator.py
+++ b/calculator.py
@@ -1,15 +1,12 @@
 class Calculator:
     def divide(self, a: float, b: float) -> float:
-        """Divide a por b com validação."""
-        if b == 0:
-            raise ValueError("Divisão por zero!")
+        """Divide a por b."""
         return a / b
 
     def calculate_average(self, numbers: list) -> float:
-        """Calcula média com validação."""
-        if not numbers:
-            return 0.0
+        """Calcula média."""
         total = sum(numbers)
-        return total / len(numbers)
+        count = len(numbers)
+        return total / count
'''

print("📝 Diff a ser analisado:")
print(diff_bugado)


📝 Diff a ser analisado:

--- a/calculator.py
+++ b/calculator.py
@@ -1,15 +1,12 @@
 class Calculator:
     def divide(self, a: float, b: float) -> float:
-        """Divide a por b com validação."""
-        if b == 0:
-            raise ValueError("Divisão por zero!")
+        """Divide a por b."""
         return a / b

     def calculate_average(self, numbers: list) -> float:
-        """Calcula média com validação."""
-        if not numbers:
-            return 0.0
+        """Calcula média."""
         total = sum(numbers)
-        return total / len(numbers)
+        count = len(numbers)
+        return total / count



In [4]:
# Analisar o diff
print("🔍 Analisando diff...\n")

resultado = reviewer.analyze_diff(
    diff=diff_bugado,
    context="Refatoração para simplificar o código da calculadora"
)

if resultado.success:
    print("📊 Análise do Diff:\n")
    print(resultado.content)
else:
    print(f"❌ Erro: {resultado.content}")


🔍 Analisando diff...

📊 Análise do Diff:

## Resumo Executivo

As alterações no código da classe `Calculator` visam simplificar as funções `divide` e `calculate_average`. No entanto, a remoção de validações essenciais introduz potenciais bugs e problemas de segurança. A simplificação do código deve ser feita com cuidado para não comprometer a robustez e a segurança das operações.

## Problemas Encontrados

1. **Bugs e Erros Potenciais**
   - **Divisão por Zero (Alta Severidade):** A remoção da verificação de divisão por zero na função `divide` pode resultar em uma exceção `ZeroDivisionError`, interrompendo a execução do programa.
   - **Cálculo de Média com Lista Vazia (Média Severidade):** A remoção da verificação de lista vazia na função `calculate_average` pode levar a uma exceção `ZeroDivisionError` quando `len(numbers)` for zero.

2. **Segurança**
   - **Validação de Entrada Insuficiente (Média Severidade):** A falta de validação de entrada pode permitir que dados inesperados ou m

## 3. Analisar Diff com Problema de Segurança

Mudança que introduz vulnerabilidade SQL Injection.


In [5]:
# Diff que introduz vulnerabilidade de segurança
diff_inseguro = '''
--- a/user_service.py
+++ b/user_service.py
@@ -5,12 +5,10 @@ class UserService:
     def __init__(self, db):
         self.db = db
 
-    def get_user(self, user_id: int) -> dict:
-        """Busca usuário de forma segura."""
-        query = "SELECT * FROM users WHERE id = ?"
-        result = self.db.execute(query, (user_id,))
+    def get_user(self, user_id) -> dict:
+        """Busca usuário."""
+        query = f"SELECT * FROM users WHERE id = {user_id}"
+        result = self.db.execute(query)
         return result.fetchone()
 
-    def search_users(self, name: str) -> list:
-        """Busca usuários por nome."""
-        query = "SELECT * FROM users WHERE name LIKE ?"
-        result = self.db.execute(query, (f"%{name}%",))
+    def search_users(self, name) -> list:
+        query = "SELECT * FROM users WHERE name LIKE '%" + name + "%'"
+        result = self.db.execute(query)
         return result.fetchall()
'''

print("⚠️ Diff com mudança de segurança:")
print(diff_inseguro)


⚠️ Diff com mudança de segurança:

--- a/user_service.py
+++ b/user_service.py
@@ -5,12 +5,10 @@ class UserService:
     def __init__(self, db):
         self.db = db

-    def get_user(self, user_id: int) -> dict:
-        """Busca usuário de forma segura."""
-        query = "SELECT * FROM users WHERE id = ?"
-        result = self.db.execute(query, (user_id,))
+    def get_user(self, user_id) -> dict:
+        """Busca usuário."""
+        query = f"SELECT * FROM users WHERE id = {user_id}"
+        result = self.db.execute(query)
         return result.fetchone()

-    def search_users(self, name: str) -> list:
-        """Busca usuários por nome."""
-        query = "SELECT * FROM users WHERE name LIKE ?"
-        result = self.db.execute(query, (f"%{name}%",))
+    def search_users(self, name) -> list:
+        query = "SELECT * FROM users WHERE name LIKE '%" + name + "%'"
+        result = self.db.execute(query)
         return result.fetchall()



In [6]:
# Analisar diff de segurança
print("🔍 Analisando diff de segurança...\n")

resultado = reviewer.analyze_diff(
    diff=diff_inseguro,
    context="PR de um desenvolvedor junior para 'simplificar' o código"
)

if resultado.success:
    print("🔒 Análise de Segurança do Diff:\n")
    print(resultado.content)
else:
    print(f"❌ Erro: {resultado.content}")


🔍 Analisando diff de segurança...

🔒 Análise de Segurança do Diff:

# Análise de Código

## Resumo Executivo

As alterações propostas pelo desenvolvedor junior introduzem sérios problemas de segurança e vão contra boas práticas de programação. As mudanças simplificam o código, mas comprometem a segurança e a robustez da aplicação.

## Problemas Encontrados

1. **Injeção de SQL (Severidade: Alta)**
   - As alterações removem o uso de parâmetros preparados, substituindo-os por interpolação direta de strings. Isso torna o código vulnerável a ataques de injeção de SQL.
   
2. **Remoção de Tipagem (Severidade: Média)**
   - A remoção das anotações de tipo para `user_id` e `name` reduz a legibilidade e a capacidade de manutenção do código, além de potencialmente introduzir erros de tipo.

3. **Comentário Desatualizado (Severidade: Baixa)**
   - O comentário no método `get_user` ainda menciona que a busca é feita de forma segura, o que não é mais verdade após as alterações.

## Sugestões de C

## 4. Usar com Git Real

Analisar diffs do repositório Git atual.


In [7]:
# Obter diff do último commit
print("📂 Obtendo diff do Git...\n")

# Diff entre último commit e anterior
diff_real = git_tool.get_diff("HEAD~1", "HEAD")

if diff_real:
    # Mostrar primeiros 2000 caracteres
    print(f"📝 Diff do último commit (primeiros 2000 chars):\n")
    print(diff_real[:2000])
    if len(diff_real) > 2000:
        print(f"\n... ({len(diff_real) - 2000} caracteres omitidos)")
else:
    print("⚠️ Não foi possível obter diff (pode não haver commits suficientes)")


📂 Obtendo diff do Git...

⚠️ Não foi possível obter diff (pode não haver commits suficientes)


In [8]:
# Analisar diff real (se existir)
if diff_real and len(diff_real) > 0:
    print("🔍 Analisando diff do Git...\n")
    
    # Limitar tamanho para não estourar contexto
    diff_para_analise = diff_real[:10000] if len(diff_real) > 10000 else diff_real
    
    resultado = reviewer.analyze_diff(
        diff=diff_para_analise,
        context="Último commit do repositório"
    )
    
    if resultado.success:
        print("📊 Análise do Commit:\n")
        print(resultado.content)
    else:
        print(f"❌ Erro: {resultado.content}")
else:
    print("⚠️ Nenhum diff disponível para análise")


⚠️ Nenhum diff disponível para análise


## 5. Comparar Versões de Código

Compare duas versões de código diretamente.


In [9]:
import difflib

# Versão antiga do código
codigo_antigo = '''
def processar_pedido(pedido):
    """Processa um pedido."""
    if pedido is None:
        raise ValueError("Pedido não pode ser nulo")
    
    if not pedido.get("items"):
        raise ValueError("Pedido deve ter itens")
    
    total = 0
    for item in pedido["items"]:
        total += item["preco"] * item["quantidade"]
    
    if pedido.get("desconto"):
        total = total * (1 - pedido["desconto"] / 100)
    
    return {
        "total": total,
        "status": "processado"
    }
'''

# Versão nova do código (com problemas)
codigo_novo = '''
def processar_pedido(pedido):
    """Processa um pedido."""
    total = 0
    for item in pedido["items"]:
        total += item["preco"] * item["quantidade"]
    
    total = total * (1 - pedido["desconto"] / 100)
    
    return {
        "total": total,
        "status": "processado"
    }
'''

# Gerar diff
diff_gerado = difflib.unified_diff(
    codigo_antigo.splitlines(keepends=True),
    codigo_novo.splitlines(keepends=True),
    fromfile='antes.py',
    tofile='depois.py'
)

diff_texto = ''.join(diff_gerado)
print("📝 Diff gerado automaticamente:")
print(diff_texto)


📝 Diff gerado automaticamente:
--- antes.py
+++ depois.py
@@ -1,18 +1,11 @@
 
 def processar_pedido(pedido):
     """Processa um pedido."""
-    if pedido is None:
-        raise ValueError("Pedido não pode ser nulo")
-
-    if not pedido.get("items"):
-        raise ValueError("Pedido deve ter itens")
-
     total = 0
     for item in pedido["items"]:
         total += item["preco"] * item["quantidade"]
 
-    if pedido.get("desconto"):
-        total = total * (1 - pedido["desconto"] / 100)
+    total = total * (1 - pedido["desconto"] / 100)
 
     return {
         "total": total,



In [10]:
# Analisar o diff gerado
print("🔍 Analisando mudanças...\n")

resultado = reviewer.analyze_diff(
    diff=diff_texto,
    context="Refatoração para 'simplificar' processamento de pedidos"
)

if resultado.success:
    print("📊 Análise da Refatoração:\n")
    print(resultado.content)
else:
    print(f"❌ Erro: {resultado.content}")


🔍 Analisando mudanças...

📊 Análise da Refatoração:

## Resumo Executivo

A refatoração do código visa simplificar o processamento de pedidos, mas introduz problemas significativos que podem afetar a funcionalidade e a robustez do sistema. A remoção de verificações de nulidade e de itens no pedido pode resultar em exceções não tratadas e cálculos incorretos. Além disso, a alteração não melhora a legibilidade ou a manutenibilidade do código.

## Problemas Encontrados

1. **Bugs e Erros Potenciais**
   - **Severidade Alta**: A remoção das verificações para `pedido is None` e `pedido.get("items")` pode levar a exceções `TypeError` ou `KeyError` se o pedido for `None` ou não contiver a chave "items".
   - **Severidade Média**: A remoção da verificação de existência do desconto pode resultar em um `TypeError` se a chave "desconto" não estiver presente no pedido.

2. **Segurança**
   - **Severidade Baixa**: Não foram identificados problemas de segurança diretamente relacionados às mudanças.


## 6. Resumo

A análise de diff é útil para:

| Caso de Uso | Descrição |
|-------------|-----------|
| **Code Review** | Revisar PRs antes de merge |
| **CI/CD** | Validar mudanças automaticamente |
| **Segurança** | Detectar vulnerabilidades introduzidas |
| **Qualidade** | Identificar bugs em novas mudanças |

### Métodos Disponíveis

| Método | Descrição |
|--------|-----------|
| `reviewer.analyze_diff()` | Analisa um diff/patch |
| `git_tool.get_diff()` | Obtém diff entre commits |
| `git_tool.get_staged_diff()` | Obtém diff dos arquivos staged |

### Integração com Git

```python
# Analisar último commit
diff = git_tool.get_diff("HEAD~1", "HEAD")
resultado = reviewer.analyze_diff(diff)

# Analisar arquivos staged (antes do commit)
diff = git_tool.get_staged_diff()
resultado = reviewer.analyze_diff(diff)

# Analisar arquivo específico
diff = git_tool.get_diff("main", "feature", file_path="src/main.py")
resultado = reviewer.analyze_diff(diff)
```


In [11]:
print("🎉 Demo concluída!")
print("\n💡 Dicas para Code Review:")
print("  1. Sempre revise diffs antes de fazer merge")
print("  2. Use análise automática em PRs no CI/CD")
print("  3. Preste atenção em validações removidas")
print("  4. Verifique se testes foram atualizados")
print("  5. Cuidado com mudanças que 'simplificam' código de segurança")


🎉 Demo concluída!

💡 Dicas para Code Review:
  1. Sempre revise diffs antes de fazer merge
  2. Use análise automática em PRs no CI/CD
  3. Preste atenção em validações removidas
  4. Verifique se testes foram atualizados
  5. Cuidado com mudanças que 'simplificam' código de segurança
